In [ ]:
import numpy as np
import pandas as pd
import time
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry
from datetime import datetime, timedelta
import random
import math
import gesla 

In [2]:
flood_df = pd.read_csv("../1 Geo coding/src/HANZE_events.csv")

In [3]:
flood_df.head()

,ID,Country code,Year,Country name,Start date,End date,Type,Flood source,Regions affected (v2010),Regions affected (v2021),Area flooded,Fatalities,Persons affected,Losses (nominal value),Losses (original currency),Losses (2020 euro),Cause,Notes,References,Changes
0,2000,AL,1946,Albania,1946-10-11,1946-11-15,River,NaN,AL011;AL012;AL013;AL015;AL021;AL022;AL031;AL03...,AL011;AL012;AL013;AL015;AL021;AL022;AL031;AL03...,NaN,15.0,NaN,1.000000e+09,ALK,29746037.0,Heavy rainfall,Also landslides,UNDRR (2022),New
1,2001,AL,1953,Albania,1953-12-12,1953-12-12,Flash,NaN,AL014;AL015,AL014;AL015,NaN,5.0,NaN,8.000000e+07,ALK,2379683.0,Extreme rainfall,NaN,UNDRR (2022),New
2,2002,AL,1960,Albania,1960-11-07,1960-11-07,Flash,NaN,AL022;AL032;AL035,AL022;AL032;AL035,NaN,4.0,NaN,NaN,NaN,NaN,Extreme rainfall,NaN,UNDRR (2022),New
3,2003,AL,1962,Albania,1962-11-15,1962-11-16,River,Drin; Buna; Semani; Vjosa,AL011;AL012;AL013;AL014;AL015;AL021;AL022;AL03...,AL011;AL012;AL013;AL014;AL015;AL021;AL022;AL03...,995.0,9.0,NaN,2.000000e+09,ALK,59492073.0,Heavy rainfall,Flood impact continued into 1963; Impact area ...,UNDRR (2022); GIZ (2018); Bogdani and Selenica...,New
4,2532,AL,1968,Albania,1968-12-17,1968-12-17,Flash,NaN,AL015,AL015,15.0,0.0,NaN,NaN,NaN,NaN,Extreme rainfall,NaN,UNDRR (2022),New


In [4]:
flood_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2521 entries, 0 to 2520
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          2521 non-null   int64  
 1   Country code                2521 non-null   object 
 2   Year                        2521 non-null   int64  
 3   Country name                2521 non-null   object 
 4   Start date                  2521 non-null   object 
 5   End date                    2521 non-null   object 
 6   Type                        2521 non-null   object 
 7   Flood source                1647 non-null   object 
 8   Regions affected (v2010)    2521 non-null   object 
 9   Regions affected (v2021)    2521 non-null   object 
 10  Area flooded                394 non-null    float64
 11  Fatalities                  2496 non-null   float64
 12  Persons affected            1096 non-null   float64
 13  Losses (nominal value)      1005 

In [5]:
flood_df = flood_df[flood_df["Year"] >= 2005]

flood_df['Start date'] = pd.to_datetime(flood_df['Start date'])
flood_df['End date'] = pd.to_datetime(flood_df['End date'])

In [6]:
regions_code_df = pd.read_csv("../1 Geo coding/output/regions_codes_names_and_coords.csv")
regions_code_df.head(1)

,Code,Name,Country code,Country name,Latitude,Longitude,Sea distance,Sea latitude,Sea longitude
0,AD000,Andorra,AD,Andorra,42.506285,1.521801,120770.942754,42.473083,2.986084


In [7]:
flood_df
flood_df_regions = pd.DataFrame(columns=["lat", "lon", "Start date", "End date", "Sea distance", "Sea latitude", "Sea longitude", "Type"])
for index, row in flood_df.iterrows():  
    region_codes = row["Regions affected (v2021)"]
    for region_code in region_codes.split(";") :
        if region_code.startswith("DEC"):
            parts = region_code.split('C')
            region_code = "Dec-" + parts[1]
            
        region_coord = regions_code_df[regions_code_df["Code"] == region_code]

        flood_df_regions.loc[len(flood_df_regions)] = {
            'lat': region_coord["Latitude"].iloc[0], 
            'lon': region_coord["Longitude"].iloc[0], 
            'Sea distance' : region_coord["Sea distance"].iloc[0], 
            'Sea latitude' : region_coord["Sea latitude"].iloc[0], 
            'Sea longitude' : region_coord["Sea longitude"].iloc[0], 
            'Start date': row['Start date'], 
            'End date' : row['End date'],
            'Type' : row["Type"]}



In [8]:
flood_df_regions.head()

,lat,lon,Start date,End date,Sea distance,Sea latitude,Sea longitude,Type
0,41.324590,19.456469,2005-11-30,2005-12-03,5300.276600,41.329600,19.393639,River
1,41.786073,19.646076,2005-11-30,2005-12-03,11519.642694,41.721554,19.538546,River
2,41.111384,20.082263,2005-11-30,2005-12-03,59968.278192,41.170201,19.373702,River
3,40.705251,19.951891,2005-11-30,2005-12-03,52905.672506,40.645285,19.334144,River
4,40.727870,19.559631,2005-11-30,2005-12-03,20235.680752,40.704452,19.323288,River


In [592]:
flood_df_regions.to_csv("output/flood_df_regions.csv", index=False)

In [595]:
meta_file = "src/GESLA3_ALL 2.csv"
data_path = "src/GESLA3/"
g3 = gesla.GeslaDataset(meta_file=meta_file, data_path=data_path)

In [599]:
g3.meta["end_date_time"]

0       1999-06-28 11:18:00
1       1999-06-20 23:53:00
2       2018-12-31 23:00:00
3       2014-12-31 23:00:00
4       2018-12-31 23:00:00
               ...         
5114    2018-12-31 23:00:00
5115    2013-12-30 11:00:00
5116    2013-12-30 11:00:00
5117    2011-12-31 23:00:00
5118    2012-12-31 23:00:00
Name: end_date_time, Length: 5119, dtype: object

In [610]:
Gesla_meta_light = g3.meta[g3.meta["end_date_time"] >= datetime(2005,1,1) ]

In [611]:
lon_bool = (Gesla_meta_light.longitude >= -30) & (
                Gesla_meta_light.longitude <= 40
            )
lat_bool = (Gesla_meta_light.latitude >= 30) & (Gesla_meta_light.latitude <= 67)
Gesla_meta_light = Gesla_meta_light.loc[lon_bool & lat_bool]

In [616]:
Gesla_meta_light = Gesla_meta_light[Gesla_meta_light["overall_record_quality"] == "No obvious issues"]
Gesla_meta_light.reset_index(drop=True, inplace=True)

In [618]:
Gesla_meta_light.to_csv("output/GESLA3_light.csv")

In [652]:
for filename in Gesla_meta_light["filename"] :
    print(filename)
    sea_level_df = g3.file_to_pandas(filename=filename, return_meta=False)
    sea_level_df = sea_level_df[sea_level_df.index >= datetime(2005,1,1)].copy()
    sea_level_df = sea_level_df.resample("h").median() # median to minimize the impact of outlier
    sea_level_df.dropna(inplace=True)
    sea_level_df.to_csv(f"output/GESLA3_light/{filename}")

brest-822a-fra-uhslc
stockholm-826a-swe-uhslc
hornbaek-838a-dnk-uhslc
gedser-837a-dnk-uhslc
marseille-824a-fra-uhslc
cuxhaven-825a-deu-uhslc
newlyn_cornwall-294a-gbr-uhslc
tregde-804a-nor-uhslc
la_coruna-830a-esp-uhslc
ceuta-207a-esp-uhslc
lerwick-293a-gbr-uhslc
rorvik-803a-nor-uhslc
goteborgorsh-819a-swe-uhslc
cascais-209a-prt-uhslc
gibraltar-289a-gbr-uhslc
stornoway-295a-gbr-uhslc
ponta_delgada-211a-prt-uhslc
funchal-218b-prt-uhslc
maloy-802a-nor-uhslc
alexandria-807a-egy-uhslc
castletownsend-835a-irl-uhslc
newlyn-new-gbr-bodc
aberdeen-abe-gbr-bodc
dover-dov-gbr-bodc
north_shields-nsh-gbr-bodc
immingham-imm-gbr-bodc
lerwick-ler-gbr-bodc
milford_haven-mha-gbr-bodc
fishguard-fis-gbr-bodc
lowestoft-low-gbr-bodc
heysham-hey-gbr-bodc
wick-wic-gbr-bodc
portpatrick-por-gbr-bodc
ullapool-ull-gbr-bodc
holyhead-hol-gbr-bodc
sheerness-she-gbr-bodc
ilfracombe-ilf-gbr-bodc
stornoway-sto-gbr-bodc
whitby-whi-gbr-bodc
avonmouth-avo-gbr-bodc
millport-mil-gbr-bodc
newhaven-nha-gbr-bodc
cromer-cro-gbr-

In [660]:
meta_file = "output/GESLA3_light.csv"
data_path = "output/GESLA3_light/"
g3 = gesla.GeslaDataset(meta_file=meta_file, data_path=data_path)

In [655]:
pd.read_csv(f"output/GESLA3_light/{Gesla_meta_light["filename"].iloc[0]}")

,datetime,sea_level,qc_flag,use_flag
0,2005-01-01 00:00:00,2.510,1.0,1.0
1,2005-01-01 01:00:00,2.140,1.0,1.0
2,2005-01-01 02:00:00,2.310,1.0,1.0
3,2005-01-01 03:00:00,3.030,1.0,1.0
4,2005-01-01 04:00:00,3.980,1.0,1.0
...,...,...,...,...
122707,2018-12-31 19:00:00,2.520,1.0,1.0
122708,2018-12-31 20:00:00,3.191,1.0,1.0
122709,2018-12-31 21:00:00,3.995,1.0,1.0
122710,2018-12-31 22:00:00,4.790,1.0,1.0


In [522]:
delta_30_days = timedelta(days=30)
delta_1_day = timedelta(days=1)
delta_23_h = timedelta(hours=23,minutes=59)
def Create_df(data, start_date, end_date, day_time=24):
    result_df = pd.DataFrame()
    day = start_date
    i = 0
    while day <= end_date :
        # Define offsets in hours
        start_30 = i * day_time
        end = start_30 + 31 * day_time # 31 because open meteo end date is inclusive
        start_5 = end - 5 * day_time  # Last 5 days of the 30-day window
        start_1 = end - 1 * day_time  # Last 1 day of the 30-day window
        
        slice_30 = slice(start_30, end)
        slice_5 = slice(start_5, end)
        slice_1 = slice(start_1, end)

        new_row = {
            f"median_{key}_30": np.nanmedian(value[slice_30])
            for key, value in data.items()
        } | {
            f"mean_{key}_{suffix}": np.nanmean(value[slice_])
            for suffix, slice_ in [
                ("30", slice_30),
                ("5", slice_5),
                ("1", slice_1)
            ]
            for key, value in data.items()
        } | {
            f"max_{key}_1": np.nanmax(value[slice_1])
            for key, value in data.items()
        } 
        new_row["date"]= day

        if len(result_df) == 0 :
            result_df = pd.DataFrame([new_row])
        else :
            result_df.loc[len(result_df)] = new_row
        day = day + delta_1_day
        i += 1
    return result_df

def Get_previous_month_weather(lat, lon, start_date, end_date) :

    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": (start_date - delta_30_days).strftime('%Y-%m-%d'),
        "end_date": end_date.strftime('%Y-%m-%d'),
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_gusts_10m", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
        "timezone": "auto"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    elevation = response.Elevation()

    # Process daily data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(4).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(5).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(7).ValuesAsNumpy()

    hourly_soil_moisture_0_to_7cm = hourly.Variables(8).ValuesAsNumpy()
    hourly_soil_moisture_7_to_28cm = hourly.Variables(9).ValuesAsNumpy()
    hourly_soil_moisture_28_to_100cm = hourly.Variables(10).ValuesAsNumpy()
    hourly_soil_moisture_100_to_255cm = hourly.Variables(11).ValuesAsNumpy()


    weather_data = {}

    weather_data["temperature_2m"] = hourly_temperature_2m
    weather_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    weather_data["dew_point_2m"] = hourly_dew_point_2m
    weather_data["precipitation"] = hourly_precipitation

    weather_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    weather_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    weather_data["wind_speed_10m"] = hourly_wind_speed_10m
    weather_data["wind_gusts_10m"] = hourly_wind_gusts_10m


    result_df = Create_df(weather_data, start_date,end_date)

    weather_data = {} # Isolating soil moisture to preserve the feature order with forcast data.

    weather_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
    weather_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
    weather_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
    weather_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm

    result_df2 = Create_df(weather_data, start_date,end_date)


    result_df = pd.merge(result_df, result_df2,on=["date"])
    result_df["elevation"] = elevation
    result_df["lat"] = lat
    result_df["lon"] = lon
    return result_df

# Function to get river flow data from USGS API
def get_river_discharge(lat, lon, start_date, end_date):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://flood-api.open-meteo.com/v1/flood"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "river_discharge",
        "start_date": (start_date - delta_30_days).strftime('%Y-%m-%d'),
        "end_date": end_date.strftime('%Y-%m-%d'),
        "models": "seamless_v4",
        "timezone": "auto"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_river_discharge = daily.Variables(0).ValuesAsNumpy()
    river_data = {"river_discharge" : daily_river_discharge}
    result_df = Create_df(river_data, start_date,end_date,1)
    result_df["lat"] = lat
    result_df["lon"] = lon
    return result_df

def Get_marine_weather(lat, lon, sea_lat, sea_lon, start_date, end_date):
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=2, backoff_factor=0.1)  
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://marine-api.open-meteo.com/v1/marine"
    max_attempts = 10  # Maximum coordinate adjustment attempts
    attempt = 0
    original_sea_lat, original_sea_lon = sea_lat, sea_lon  # Save original coordinates
    params = {
                "hourly": "wave_height",
                "models": "era5_ocean",
                "timezone": "GMT",
                "start_date": (start_date - delta_30_days).strftime('%Y-%m-%d'),
                "end_date": end_date.strftime('%Y-%m-%d'),
            }
    
    # Calculate displacement vector
    delta_lat = lat - sea_lat
    delta_lon = lon - sea_lon
    norm = math.hypot(delta_lat, delta_lon)
    while attempt < max_attempts:
        try:
            
            params["latitude"] = sea_lat
            params["longitude"] = sea_lon
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]  

            # Process successful response
            hourly = response.Hourly()
            hourly_wave_height = hourly.Variables(0).ValuesAsNumpy()
            
            weather_data = {"wave_height": hourly_wave_height}
            result_df = Create_df(weather_data, start_date, end_date)
            result_df["lat"] = lat
            result_df["lon"] = lon
            return result_df

        except Exception as e:
            print(f"Attempt {attempt+1} failed: {str(e)}")

            # Handle zero-length vector (same coordinates)
            if norm == 0:
                sea_lat += 0.1  # Small adjustment
                sea_lon += 0.1
            else:
                # Move 10% further away each attempt (adjust multiplier as needed)
                adjustment = 0.1 * (attempt + 1)
                sea_lat -= (delta_lat / norm) * adjustment
                sea_lon -= (delta_lon / norm) * adjustment

            attempt += 1

    # If all attempts fail, restore original coordinates for final error
    raise RuntimeError(
        f"Failed after {max_attempts} attempts. Last tried coordinates: "
        f"{sea_lat:.3f}, {sea_lon:.3f} (original: {original_sea_lat}, {original_sea_lon})"
    )

In [ ]:
def merge_duplicates(group):
    # Sort by station index
    
    # Filter for rows with use_flag == 1 first 
    # Use-in-analysis flag (1 = use, 0 = do not use)
    use_flag_group = group[group["use_flag"] == 1]
    # Priority checks
    # 0 - no quality control
    # 1 - correct value 
    # 2 - interpolated value
    # 3 - doubtful value
    # 4 - isolated spike or wrong value
    # 5 - missing value
    for qc_value in [1, 2, 3, 0]:  # Ordered by priority
        if (use_flag_group["qc_flag"] == qc_value).any():
            return use_flag_group.loc[use_flag_group["qc_flag"] == qc_value].iloc[0]
    # Fallback priorities (check all rows)
    for qc_value in [1, 2, 3, 0, 4]:  # Ordered by priority
        if (group["qc_flag"] == qc_value).any():
            return group.loc[group["qc_flag"] == qc_value].iloc[0]
    # Final fallbacks
    if (group["use_flag"] == 1).any():
        return group.loc[group["use_flag"] == 1].median() 
    
    return group.median()  # just take median 

def Get_sea_level(lat,lon,sea_lat, sea_lon, start_date, end_date, N=10) :
    data_start_date = start_date - delta_30_days
    data_end_date = end_date + delta_23_h
    filenames = g3.load_N_closest(lat=sea_lat, lon=sea_lon, N=N,  start_date=data_start_date, end_date=data_end_date, return_only_filename_list=True)
    data = pd.DataFrame()
    for filename in filenames :
        file_df = pd.read_csv("output/GESLA3_light/" + filename)
        file_df['datetime'] = pd.to_datetime(file_df['datetime'])
        file_df = file_df[(file_df['datetime'] >= data_start_date) & (file_df['datetime'] <= data_end_date)]
        data = pd.concat([data, file_df]).reset_index(drop=True)
    data = (
        data.groupby("datetime")
        .apply(merge_duplicates, include_groups=False)
    )
    sea_level = data["sea_level"].copy()
    # Add zeros at start_date and end_date
    # Add 'start_date' if it doesn't exist in the DataFrame index
    if data_start_date not in sea_level.index:
        sea_level[data_start_date] = 0

    # Add 'end_date + delta_23_h' if it doesn't exist in the DataFrame index
    if (data_end_date) not in sea_level.index:
        sea_level[data_end_date] = 0
    sea_level = sea_level.sort_index()

    # complete missing values 
    sea_level = sea_level.resample('h').interpolate("linear") 

    weather_data = {}
    weather_data["sea_level_height_msl"] = sea_level.to_numpy()
    result_df = Create_df(weather_data, start_date,end_date)
    result_df["lat"] = lat
    result_df["lon"] = lon
    return result_df

In [ ]:
flood_weather_data = pd.DataFrame()

for index, row in flood_df_regions.iterrows():
    row_sea_level = Get_sea_level(lat=row["lat"], lon=row["lon"],sea_lat=row["Sea latitude"], sea_lon=row["Sea longitude"],  start_date=row["Start date"], end_date=row["End date"])
    row_sea_level["Sea distance"] = row["Sea distance"]
    row_sea_level["Type"] = row["Type"]
    flood_weather_data = pd.concat([flood_weather_data, row_sea_level], ignore_index=True)

In [671]:
flood_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20830 entries, 0 to 20829
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   median_sea_level_height_msl_30  20830 non-null  float64       
 1   mean_sea_level_height_msl_30    20830 non-null  float64       
 2   mean_sea_level_height_msl_5     20830 non-null  float64       
 3   mean_sea_level_height_msl_1     20830 non-null  float64       
 4   max_sea_level_height_msl_1      20830 non-null  float64       
 5   date                            20830 non-null  datetime64[ns]
 6   lat                             20830 non-null  float64       
 7   lon                             20830 non-null  float64       
 8   Sea distance                    20830 non-null  float64       
 9   Type                            20830 non-null  object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 1.6+ MB


In [672]:
flood_weather_data.to_csv("output/flood_and_weather.csv", index=False)

In [341]:
flood_weather_data_2 = pd.DataFrame()

In [375]:
flood_weather_data

,median_sea_level_height_msl_30,mean_sea_level_height_msl_30,mean_sea_level_height_msl_5,mean_sea_level_height_msl_1,max_sea_level_height_msl_1,date,lat,lon,Sea distance,Type
0,-0.015833,-0.014045,0.079077,0.072226,0.155300,2005-11-30,41.324590,19.456469,5300.276600,River
1,-0.014175,-0.010989,0.065424,0.040837,0.184400,2005-12-01,41.324590,19.456469,5300.276600,River
2,-0.012925,-0.008916,0.047648,0.021365,0.126400,2005-12-02,41.324590,19.456469,5300.276600,River
3,-0.006250,-0.005174,0.056531,0.098718,0.181967,2005-12-03,41.324590,19.456469,5300.276600,River
4,0.087517,0.096306,0.256133,0.234795,0.421967,2005-11-30,41.786073,19.646076,11519.642694,River
...,...,...,...,...,...,...,...,...,...,...
20825,3.395313,3.443315,3.593709,3.675031,5.098500,2020-12-24,51.761206,-1.246467,220333.120686,River
20826,3.389250,3.439519,3.579144,3.363719,4.899500,2020-12-25,51.761206,-1.246467,220333.120686,River
20827,3.381875,3.433660,3.535774,3.302370,5.004250,2020-12-26,51.761206,-1.246467,220333.120686,River
20828,3.381875,3.437727,3.495685,3.530219,5.829875,2020-12-27,51.761206,-1.246467,220333.120686,River


In [ ]:
chunk = flood_df_regions.iloc[1100:] #chunk per chunk to respect open meteo limitations (5000 per hour, 10k per day)

for index, row in chunk.iterrows():
   start_time = time.time()
   row_marine_weather = Get_marine_weather(lat=row["lat"], lon=row["lon"],sea_lat=row["Sea latitude"], sea_lon=row["Sea longitude"],  start_date=row["Start date"], end_date=row["End date"]) # ~ 1 call
   row_river_discharge = get_river_discharge(lat=row["lat"], lon=row["lon"], start_date=row["Start date"], end_date=row["End date"]) # ~ 1 calls
   row_weather = Get_previous_month_weather(lat=row["lat"], lon=row["lon"], start_date=row["Start date"], end_date=row["End date"]) # ~ 2 call
   

   complete_df = pd.merge(row_river_discharge, row_weather,on=["date", "lat", "lon"])
   complete_df = pd.merge(complete_df, row_marine_weather,on=["date", "lat", "lon"])
   flood_weather_data_2 = pd.concat([flood_weather_data_2,complete_df],ignore_index=True)
   elapsed_time = time.time() - start_time
   if(elapsed_time < 0.6) : 
      time.sleep(0.6 - elapsed_time) # 400 call per minute to respect the open meteo limitations (600)

C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:20: Ru

Attempt 1 failed: {'error': True, 'reason': 'No data is available for this location'}


C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\872636786.py:20: Ru

In [365]:
flood_weather_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20830 entries, 0 to 20829
Data columns (total 74 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   median_river_discharge_30             20544 non-null  float32       
 1   mean_river_discharge_30               20544 non-null  float32       
 2   mean_river_discharge_5                20544 non-null  float32       
 3   mean_river_discharge_1                20544 non-null  float32       
 4   max_river_discharge_1                 20544 non-null  float32       
 5   date                                  20830 non-null  datetime64[ns]
 6   lat                                   20830 non-null  float64       
 7   lon                                   20830 non-null  float64       
 8   median_temperature_2m_30              20830 non-null  float32       
 9   median_relative_humidity_2m_30        20830 non-null  float32       
 10

In [366]:
flood_weather_data_2.to_csv("output/flood_and_weather_2.csv", index=False)

In [673]:
common_cols = flood_weather_data.columns.intersection(flood_weather_data_2.columns).tolist()
df1 = flood_weather_data.drop_duplicates(subset=common_cols, keep='first')
df2 = flood_weather_data_2.drop_duplicates(subset=common_cols, keep='first')
complete_flood_weather_data= pd.merge(df1, df2, on=common_cols)

In [674]:
complete_flood_weather_data.to_csv("output/complete_flood_weather_data.csv", index=False)

In [ ]:

def generate_random_date(start_date, end_date):
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

def generate_new_dates(existing_dates, n_dates, min_gap, min_date, max_date, max_tries=10):
    new_dates = []
    for _ in range(n_dates):
        for _ in range(max_tries):
            new_date = generate_random_date(min_date, max_date)
            if not any(abs((new_date - date).days) < min_gap * 30 for date in existing_dates):
                break
        else:  # If all tries failed, add min gap to random_start_date
            new_date = min_date
            while any(abs((new_date - date).days) < min_gap * 30 for date in existing_dates):
                new_date += timedelta(days=min_gap * 30)
        new_dates.append(new_date)
        existing_dates.append(new_date)
        max_date = new_date
    return  new_dates

In [401]:
regions_code_df

,Code,Name,Country code,Country name,Latitude,Longitude,Sea distance,Sea latitude,Sea longitude
0,AD000,Andorra,AD,Andorra,42.506285,1.521801,120770.942754,42.473083,2.986084
1,AL011,Dibër,AL,Albania,41.588816,20.235565,59744.477832,41.720093,19.540100
2,AL012,Durrës,AL,Albania,41.324590,19.456469,5300.276600,41.329600,19.393639
3,AL013,Kukës,AL,Albania,42.075584,20.425946,83896.952278,41.720093,19.540100
4,AL014,Lezhë,AL,Albania,41.786073,19.646076,11519.642694,41.721554,19.538546
...,...,...,...,...,...,...,...,...,...
1438,XK006,Gjilan/Gnjilane,XK,Kosovo,42.388746,21.427589,173918.837445,41.720093,19.540100
1439,XK002,Mitrovicë/Mitrovica,XK,Kosovo,42.994799,20.807608,172101.299510,41.877686,19.371887
1440,XK003,Pejë/Peć,XK,Kosovo,42.720918,20.473876,131094.251710,41.877686,19.371887
1441,XK001,Prishtinë/Priština,XK,Kosovo,42.669461,21.141423,170035.123026,41.729881,19.529686


In [407]:
for loc, group in complete_flood_weather_data.groupby(['lat', 'lon']):
    group['date']

In [417]:
new_rows = []
max_date = max(complete_flood_weather_data["date"].tolist())
min_date = datetime(2015,1,1)
timedelta_60 = timedelta(days=60)
for loc, group in complete_flood_weather_data.groupby(['lat', 'lon']):
    existing_dates = list(group['date'])
    new_dates = generate_new_dates(existing_dates, 2, 5, min_date, max_date)
    region_sea_coord = regions_code_df[(regions_code_df["Latitude"] == loc[0]) & (regions_code_df["Longitude"] == loc[1])]
    for date in new_dates:
        new_rows.append({
            'lat': loc[0], 
            'lon': loc[1], 
            'Sea distance':region_sea_coord["Sea distance"].iloc[0], 
            'Sea latitude':region_sea_coord["Sea latitude"].iloc[0], 
            'Sea longitude':region_sea_coord["Sea longitude"].iloc[0], 
            'Start date': date, 
            'End date' : date + timedelta_60, 
            'Type': 'no flood'})

In [418]:
len(new_rows)

1668

In [419]:
no_flood_rows_df = pd.DataFrame(new_rows)

In [675]:
no_flood_weather_data = pd.DataFrame()
for index, row in no_flood_rows_df.iterrows():
    row_sea_level = Get_sea_level(lat=row["lat"], lon=row["lon"],sea_lat=row["Sea latitude"], sea_lon=row["Sea longitude"],  start_date=row["Start date"], end_date=row["End date"])
    row_sea_level["Sea distance"] = row["Sea distance"]
    row_sea_level["Type"] = row["Type"]
    no_flood_weather_data = pd.concat([no_flood_weather_data, row_sea_level], ignore_index=True)

In [676]:
no_flood_weather_data.to_csv("output/no_flood_and_weather.csv",index=False)

In [539]:
no_flood_weather_data_2 = pd.DataFrame()

In [572]:
no_flood_weather_data_2

,median_river_discharge_30,mean_river_discharge_30,mean_river_discharge_5,mean_river_discharge_1,max_river_discharge_1,date,lat,lon,median_temperature_2m_30,median_relative_humidity_2m_30,...,max_soil_moisture_0_to_7cm_1,max_soil_moisture_7_to_28cm_1,max_soil_moisture_28_to_100cm_1,max_soil_moisture_100_to_255cm_1,elevation,median_wave_height_30,mean_wave_height_30,mean_wave_height_5,mean_wave_height_1,max_wave_height_1
0,0.625549,0.751508,0.592209,0.592209,0.592209,2017-06-13,35.126413,33.429859,23.275000,48.614307,...,0.148,0.133,0.105,0.178,143.0,0.42,0.525726,0.544000,0.656667,0.94
1,0.625549,0.754493,0.592209,0.592209,0.592209,2017-06-14,35.126413,33.429859,23.200001,48.842590,...,0.147,0.133,0.105,0.178,143.0,0.46,0.541156,0.599500,0.751667,0.92
2,0.625549,0.757478,0.592209,0.592209,0.592209,2017-06-15,35.126413,33.429859,23.250000,49.119644,...,0.122,0.133,0.105,0.178,143.0,0.46,0.542366,0.566833,0.458333,0.62
3,0.625549,0.759525,0.586397,0.563148,0.563148,2017-06-16,35.126413,33.429859,23.350000,49.318947,...,0.120,0.132,0.104,0.178,143.0,0.44,0.535806,0.519333,0.322500,0.38
4,0.625549,0.761572,0.580584,0.563148,0.563148,2017-06-17,35.126413,33.429859,23.400000,50.444496,...,0.141,0.131,0.104,0.178,143.0,0.44,0.537715,0.501333,0.317500,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80271,0.093956,0.226008,0.339936,0.233105,0.233105,2016-11-28,51.531456,-2.454716,6.928000,89.523697,...,0.366,0.374,0.358,0.341,58.0,1.38,1.667312,2.190167,2.330833,2.84
80272,0.101539,0.231245,0.280452,0.202264,0.202264,2016-11-29,51.531456,-2.454716,6.728000,89.166153,...,0.361,0.370,0.358,0.341,58.0,1.52,1.703495,1.933167,1.772500,2.42
80273,0.116863,0.235819,0.236900,0.172195,0.172195,2016-11-30,51.531456,-2.454716,6.578000,89.027557,...,0.359,0.367,0.358,0.341,58.0,1.52,1.689624,1.476833,0.586667,1.10
80274,0.124605,0.240104,0.205683,0.156112,0.156112,2016-12-01,51.531456,-2.454716,6.378000,88.833099,...,0.357,0.365,0.358,0.341,58.0,1.52,1.666828,1.289500,0.240000,0.28


In [577]:
no_flood_rows_df[no_flood_rows_df["lat"] == no_flood_weather_data.iloc[80276]["lat"] ]

,lat,lon,Sea distance,Sea latitude,Sea longitude,Start date,End date,Type
1316,51.536895,7.200915,217423.552226,53.4823,6.905273,2016-10-24,2016-12-23,no flood
1317,51.536895,7.200915,217423.552226,53.4823,6.905273,2015-04-27,2015-06-26,no flood


In [576]:
no_flood_weather_data.iloc[80276]

median_sea_level_height_msl_30               0.070833
mean_sea_level_height_msl_30                -0.047498
mean_sea_level_height_msl_5                  -0.04625
mean_sea_level_height_msl_1                 -0.079792
max_sea_level_height_msl_1                   0.951667
date                              2016-10-24 00:00:00
lat                                         51.536895
lon                                          7.200915
Sea distance                            217423.552226
Type                                         no flood
Name: 80276, dtype: object

In [ ]:
Get_previous_month_weather(lat=row["lat"], lon=row["lon"], start_date=row["Start date"], end_date=row["End date"])

In [578]:
chunk = no_flood_rows_df.iloc[1316:] #chunk per chunk to respect open meteo limitations

for index, row in chunk.iterrows():
   start_time = time.time()
   row_marine_weather = Get_marine_weather(lat=row["lat"], lon=row["lon"],sea_lat=row["Sea latitude"], sea_lon=row["Sea longitude"],  start_date=row["Start date"], end_date=row["End date"]) # ~ 1 call
   if(elapsed_time < 0.2) : 
      time.sleep(0.2 - elapsed_time) 
   row_river_discharge = get_river_discharge(lat=row["lat"], lon=row["lon"], start_date=row["Start date"], end_date=row["End date"]) # ~ 1 calls
   if(elapsed_time < 0.4) : 
      time.sleep(0.4 - elapsed_time) 
   row_weather = Get_previous_month_weather(lat=row["lat"], lon=row["lon"], start_date=row["Start date"], end_date=row["End date"]) # ~ 7 call
   

   complete_df = pd.merge(row_river_discharge, row_weather,on=["date", "lat", "lon"])
   complete_df = pd.merge(complete_df, row_marine_weather,on=["date", "lat", "lon"])
   no_flood_weather_data_2 = pd.concat([no_flood_weather_data_2,complete_df],ignore_index=True)
   elapsed_time = time.time() - start_time
   if(elapsed_time < 1.2) : 
      time.sleep(1.2 - elapsed_time) 

C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py

Attempt 1 failed: {'error': True, 'reason': 'No data is available for this location'}
Attempt 2 failed: {'error': True, 'reason': 'No data is available for this location'}
Attempt 3 failed: {'error': True, 'reason': 'No data is available for this location'}


C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWa

Attempt 1 failed: {'error': True, 'reason': 'No data is available for this location'}
Attempt 2 failed: {'error': True, 'reason': 'No data is available for this location'}
Attempt 3 failed: {'error': True, 'reason': 'No data is available for this location'}


C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:20: RuntimeWarning: All-NaN slice encountered
  f"median_{key}_30": np.nanmedian(value[slice_30])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:23: RuntimeWarning: Mean of empty slice
  f"mean_{key}_{suffix}": np.nanmean(value[slice_])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py:31: RuntimeWarning: All-NaN slice encountered
  f"max_{key}_1": np.nanmax(value[slice_1])
C:\Users\ad98\AppData\Local\Temp\ipykernel_42088\3928830210.py

Attempt 1 failed: {'reason': 'No data is available for this location', 'error': True}
Attempt 1 failed: {'reason': 'No data is available for this location', 'error': True}


In [579]:
no_flood_weather_data_2.to_csv("output/no_flood_and_weather_2.csv",index=False)

In [677]:
common_cols = no_flood_weather_data.columns.intersection(no_flood_weather_data_2.columns).tolist()
complete_no_flood_weather_data= pd.merge(no_flood_weather_data, no_flood_weather_data_2, on=common_cols)

In [678]:
complete_no_flood_weather_data.to_csv("output/complete_no_flood_weather_data.csv",index=False)